# Day 8 notebook

The objectives of this notebook are to practice with the concepts of

* dynamic programming
* memoization
* higher-order functions

## Knapsack problem

In today's activity, we will practice the concept of dynamic programming with a classical problem in Computer Sciences, the "Knapsack problem."  In the knapsack problem, we imagine that we are packing a knapsack (also known as a backpack) for a trip somewhere (e.g., a long backpacking trip in a remote area).  There are many items that we might like to pack in our knapsack, but it has a limited amount of space.  Suppose that there are $n$ items that we might want to pack, and the $i$th item has "size" $s_i$ and a "value" $v_i$.  The "size" is how much space the item will take up in the knapsack and the "value" is some measure of how important that item is for our trip (e.g., perhaps all of our items are food items and the value could be the number of calories in each food item).  We are given that the knapsack has a capacity $C$.

The Knapsack problem is thus to determine a subset of items to pack that 
1. can all fit in the knapsack together and 
2. maximizes the total value of the items in the knapsack.  
    
Mathematically, we wish to perform the following optimization, where $I$ is the set of $n$ items:

$\max_{I^* \subset I} \sum_{i \in I^*} v_i \\ \textrm{subject to the constraint}\ \sum_{i \in I^*} s_i \leq C$

### Solving Knapsack by dynamic programming
The Knapsack problem can be solved via a dynamic programming algorithm that breaks this problem down into the following subproblem. Let $F[capacity, num\_items]$ be the maximum value of items that could be packed into a knapsack with $capacity$ available space, only considering the first $num\_items$.  This subproblem can be solved recursively via the following recurrence:

$F[capacity, num\_items] = \left\{
\begin{array}{ll}
F[capacity, num\_items - 1], & \textrm{if } s_{num\_items} > capacity \\
\max\left\{
\begin{array}{l}
F[capacity, num\_items - 1], \\
F[capacity - s_{num\_items}, num\_items - 1] + v_{num\_items}
\end{array}\right.,& \textrm{if } s_{num\_items} \leq capacity
\end{array}\right.$

The intuition behind this recurrence is that we consider two cases for the item indexed by $num\_items$:
1.  we *exclude* this item from the knapsack (first term in the max) or
2.  we *include* this item in the knapsack, assuming it can fit in the given capacity (second term in the max)

Below we will practice with implementing a dynamic programming algorithm that uses this subproblem definition to solve the problem of finding the maximum total *value* of items that could be packed into the knapsack.  We will *not* address the related problem of determining the actual subset of items that achieves that maximum total value.

### PROBLEM 1: Recursive implementation (1 POINT)

We will start by simply implementing the solution via a recursive function, which implements the recurrence above.  This will not be an efficient algorithm, but it will be helpful to start with this as a baseline.  You are provided with some of the code already.

In [3]:
def knapsack_score_recursive(capacity, sizes, values):
    """Solves the Knapsack problem via a recursive algorithm.

    Args:
        capacity: a number giving the amount of space in the knapsack. 배낭의 총 용량, 즉 얼마나 많은 크기의 물건을 담을 수 있는지.
        sizes: a list of the size of each possible item. 각 물건의 크기를 나타내는 리스트. 
        values: a list of the value of each possible item. 각 물건의 가치를 나타내는 리스트. 
    Returns:
        A number indicating the maximum total value of items that can be packed
        in the knapsack, with each item included in the knapsack at most once.
    """

    # A local helper function that computes the Knapsack subproblem (F[capacity, num_items])
    # described above in a recursive manner. 재귀적으로 배낭 문제를 해결. 
    def subproblem(capacity, num_items):
        # base case
        # 물건이 하나도 남아 있지 않다면 (num_items == 0), 넣을 물건이 없어 배낭의 총 가치는 0. 함수는 0을 반환. 재귀 함수가 종료될 조건. 
        if num_items == 0:
            return 0
        # 크기가 너무 커서 물건을 넣을 수 없는 경우:물건을 넣지 않을 수밖에 없음. 
        # 현재 고려 중인 물건의 크기(sizes[num_items - 1])가 배낭의 남은 용량(capacity)보다 크다면, 그 물건은 넣을 수 없음. 
        elif sizes[num_items - 1] > capacity:
            return subproblem(capacity, num_items - 1)
        # 물건을 넣을 수 있는 경우: 물건은 넣지 않거나 넣거나 선택 가능. 
        else:
            # Case 1: Exclude the current item. 넣지 않기로 선택한 경우. 
            exclude_item = subproblem(capacity, num_items - 1)
            # Case 2: Include the current item. 넣기로 선택한 경우. 
            # 그 물건의 가치를 더하고 (values[num_items - 1]), 남은 용량에 대해 재귀적으로 호출하여 계산. 
            include_item = values[num_items - 1] + subproblem(capacity - sizes[num_items - 1], num_items - 1)
            # 두 경우 중에서 더 큰 값을 선택
            return max(exclude_item, include_item)

    return subproblem(capacity, len(sizes))


In [4]:
# tests for knapsack_score_recursive
test_sizes = (5, 7, 3, 2)
test_values = (7, 8, 5, 3)
assert knapsack_score_recursive(0, test_sizes, test_values) == 0
assert knapsack_score_recursive(2, test_sizes, test_values) == 3
assert knapsack_score_recursive(3, test_sizes, test_values) == 5
assert knapsack_score_recursive(5, test_sizes, test_values) == 8
assert knapsack_score_recursive(12, test_sizes, test_values) == 16
assert knapsack_score_recursive(17, test_sizes, test_values) == 23
assert knapsack_score_recursive(17, [], []) == 0
assert knapsack_score_recursive(17, [18], [1]) == 0
print("SUCCESS: knapsack_score_recursive passed all tests!")

SUCCESS: knapsack_score_recursive passed all tests!


In [5]:
#위는 동일한 하위 문제를 반복적으로 다시 계산하기 때문에 high time complexity.이를 최적화하기 위해 memoization/bottom-up dynamic programming 수행

### PROBLEM 2: Memoized implementation (1 POINT)
To implement a dynamic programming approach to this problem, we will either need to use *memoization*, or compute values to the subproblems in a *bottom-up* approach.  Let's start with the memoization strategy, as it requires minimal modification to your recursive solution above.

Recall that the memoization strategy is to store the return values of a function in a lookup table (say a Python dictionary) indexed by the argument values for the function and to simply return the value in the lookup table when the function is called again with the same arguments.  We could do this by re-writing the `subproblem` code above, but it will be cleaner to do this in a general way via a *higher-order function* that can create a memoized version of *any* function. Below is the skeleton code for such a function that will return a memoized version of its input function, `func`, which we will assume takes exactly two arguments.  Complete the code below.

In [6]:
# memoization 전략은 이전에 계산된 값을 저장해 두고, 동일 함수가 호출되면 저장된 결과를 반환함해 중복 계산을 피함. 
# 주어진 memoize_2arg_func 함수는 두 개의 인자를 받는 함수를 메모이제이션하는 역할. 

In [7]:
def memoize_2arg_func(func):
    """Creates a memoized version of a function that takes exactly two arguments.

    Args:
        func: a function that takes exactly two arguments. 두 개의 인자를 받는 함수. 
    Returns:
        A memoized version of the function func
    """
    # create a dictionary that will map arguments to return values of func
    # output은 함수 호출 결과를 딕셔너리로 저장. 키는 함수에 전달된 인자 (arg1, arg2)의 튜플로 저장되고, 값은 계산된 함수 func(arg1, arg2)의 결과. 
    outputs = dict()
    # define a new function that will be the memoized version
    # memoized는 (arg1, arg2)가 outputs 딕셔너리에 저장되어 있는지 확인하고, 저장되어 있지 않다면 해당 인자로 func를 실행하여 결과를 저장
    def memoized(arg1, arg2):
        # if the arguments are not in the dictionary, compute and store the result
        if (arg1, arg2) not in outputs:
            outputs[(arg1, arg2)] = func(arg1, arg2)
        # return the stored result
        return outputs[(arg1, arg2)]
    
    return memoized
# 작동원리
# 첫 호출: 함수가 처음 호출되면 outputs 딕셔너리에 해당 인자 (arg1, arg2)가 없기 때문에, 원래 함수 func를 호출하여 그 결과를 저장
# 두 번째 호출: 같은 인자로 다시 호출되면, outputs에 이미 결과가 저장되어 있으므로 저장된 값을 즉시 반환

In [8]:
# tests for memoize_2arg_func
num_calls_to_test_func = 0 # keep track of how many times test_func is run
# a test function that returns the sum of two numbers
def test_func(x, y):
    global num_calls_to_test_func
    num_calls_to_test_func += 1
    return x + y
test_func = memoize_2arg_func(test_func)

assert test_func(1, 2) == 3
assert num_calls_to_test_func == 1
assert test_func(2, 3) == 5
assert num_calls_to_test_func == 2
assert test_func(1, 2) == 3 # repeated call
assert num_calls_to_test_func == 2
assert test_func(2, 3) == 5 # another repeated call
assert num_calls_to_test_func == 2

num_calls_to_test2_func = 0 # keep track of how many times test2_func is run
# a second test function that returns the length k prefix of string s
def test2_func(s, k):
    global num_calls_to_test2_func
    num_calls_to_test2_func += 1
    return s[:k]
test2_func = memoize_2arg_func(test2_func)

assert test2_func("madison", 3) == "mad"
assert num_calls_to_test2_func == 1
assert test2_func("madison", 3) == "mad" # repeated call
assert num_calls_to_test2_func == 1
assert test2_func("madison", 1) == "m"
assert num_calls_to_test2_func == 2
assert test2_func("madison", 1) == "m" # another repeated call
assert num_calls_to_test2_func == 2

print("SUCCESS: memoize_2arg_func passed all tests!")

SUCCESS: memoize_2arg_func passed all tests!


Now paste in your `knapsack_score_recursive` code from above into the `knapsack_score_memoized` dynamic programming version of the algorithm below.  Note that `knapsack_score_memoized` creates a memoized version of the `subproblem` function using your `memoize_2arg_func` higher-order function. 

In [9]:
def knapsack_score_memoized(capacity, sizes, values):
    """Solves the Knapsack problem via a top-down dynamic programming algorithm.

    Args:
        capacity: a number giving the amount of space in the knapsack.
        sizes: a list of the size of each possible item
        values: a list of the value of each possible item
    Returns:
        A number indicating the maximum total value of items that can be packed
        in the knapsack, with each item included in the knapsack at most once.
    """
    
    def subproblem(capacity, num_items):
        if num_items == 0:
            return 0
        elif sizes[num_items - 1] > capacity:
            return subproblem(capacity, num_items - 1)
        else:
            # problem 1의 recursive에서처럼 물건 넣을 수 있는 경우 두가지 중 더 큰 값 선택
            return max(
                subproblem(capacity, num_items - 1),  # 넣을 수 있지만 안 넣을때
                subproblem(capacity - sizes[num_items - 1], num_items - 1) + values[num_items - 1]  # 넣을 때
            )
    
    # subproblem에 memoize 기능 추가
    subproblem = memoize_2arg_func(subproblem)
    return subproblem(capacity, len(sizes))

In [10]:
# tests for knapsack_score_memoized (not graded)
test_sizes = (5, 7, 3, 2)
test_values = (7, 8, 5, 3)
assert knapsack_score_memoized(0, test_sizes, test_values) == 0
assert knapsack_score_memoized(2, test_sizes, test_values) == 3
assert knapsack_score_memoized(3, test_sizes, test_values) == 5
assert knapsack_score_memoized(5, test_sizes, test_values) == 8
assert knapsack_score_memoized(12, test_sizes, test_values) == 16
assert knapsack_score_memoized(17, test_sizes, test_values) == 23
assert knapsack_score_memoized(17, [], []) == 0
assert knapsack_score_memoized(17, [18], [1]) == 0
print("SUCCESS: knapsack_score_memoized passed all tests!")

SUCCESS: knapsack_score_memoized passed all tests!


#### Higher-order functions available in Python
Python is a rich functional language and offers a number of higher-order functions in its standard library.  For example, the [`map`](https://docs.python.org/3/library/functions.html#map) and [`filter`](https://docs.python.org/3/library/functions.html#filter) functions are commonly-used higher-order functions that you should consider using.  Other higher-order functions are found in the [`functools`](https://docs.python.org/3/library/functools.html) module. In particular, the [functools.lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) function is a more general implementation of the memoize function that you wrote above.

For higher-order functions that return functions, "[decorator](https://docs.python.org/3/glossary.html#term-decorator)" syntax can help.  For example,

In [11]:
def test_func(x, y):
    return x + y
test_func = memoize_2arg_func(test_func)

# is more succinctly written with decorator syntax as
@memoize_2arg_func
def test_func(x, y):
    return x + y

### PROBLEM 3: Bottom-up implementation (1 POINT)
Lastly, we will implement the knapsack dynamic programming algorithm in a bottom-up style.  Here we first create a matrix that will store all of the results of the subproblems, and compute the subproblems iteratively, starting with the smallest subproblems first.  Below is some partially completed code that you are to fill in.

*Note: you should not make any calls to knapsack_score_recursive or knapsack_score_memoized in your solution.  You should simply be calculating entries in the matrix based on other previously calculated entries in the matrix.*

In [ ]:
# 수업중에 배운 computational approach matrix랑 같은 원리

In [12]:
def knapsack_score_bottomup(capacity, sizes, values):
    """Solves the Knapsack problem via a bottom-up dynamic programming algorithm.

    Args:
        capacity: a number giving the amount of space in the knapsack.
        sizes: a list of the size of each possible item
        values: a list of the value of each possible item
    Returns:
        A number indicating the maximum total value of items that can be packed
        in the knapsack, with each item included in the knapsack at most once.
    """
    # inititialize a matrix (list of lists) to store the results of the subproblem F[num_items, capacity]
    # 행은 0부터 capacity까지의 용량을 나타내고 열은 0부터 len(sizes)까지 고려되는 항목 수를 나타내는 2D 행렬(matrix)이 생성
    matrix = [[None] * (len(sizes) + 1) for i in range(capacity + 1)]
    
    # 영상에서 한 것처럼 첫 번째 행렬부터 쭉 채우기. 이건 열. 
    for j in range(len(sizes) + 1):
        matrix[0][j] = 0
        
    # 이건 행.
    for i in range(capacity + 1):
        matrix[i][0] = 0
        
    # fill in remaining elements. 나머지 순차적으로 채워넣기. 
    for i in range(1, capacity + 1):
        for j in range(1, len(sizes) + 1):
            # knapsack에 들어가는지 사이즈 확인
            if sizes[j - 1] <= i:
                # 들어가긴 들어가는데 넣을지 안넣을지 중에서 max 값
                matrix[i][j] = max(
                    matrix[i][j - 1],  # Exclude
                    matrix[i - sizes[j - 1]][j - 1] + values[j - 1]  # Include
                )
            else:
                # 맞지 않는 경우에는 이 항목이 없는 값만 사용
                matrix[i][j] = matrix[i][j - 1]
    
    # 마지막으로 행렬의 오른쪽 하단에 있는 값(matrix[capacity][len(sizes)])은 주어진 용량과 항목으로 달성할 수 있는 최대값
    return matrix[capacity][len(sizes)]

In [13]:
# tests for knapsack_score_bottomup
test_sizes = (5, 7, 3, 2)
test_values = (7, 8, 5, 3)
assert knapsack_score_bottomup(0, test_sizes, test_values) == 0
assert knapsack_score_bottomup(2, test_sizes, test_values) == 3
assert knapsack_score_bottomup(3, test_sizes, test_values) == 5
assert knapsack_score_bottomup(5, test_sizes, test_values) == 8
assert knapsack_score_bottomup(12, test_sizes, test_values) == 16
assert knapsack_score_bottomup(17, test_sizes, test_values) == 23
assert knapsack_score_bottomup(17, [], []) == 0
assert knapsack_score_bottomup(17, [18], [1]) == 0
print("SUCCESS: knapsack_score_bottomup passed all tests!")

SUCCESS: knapsack_score_bottomup passed all tests!


### Comparing performance of the three implementations

With your three implementations of an algorithm to solve the knapsack problem, let's test their runtime to see how they perform relative to each other.

In [14]:
import timeit
capacity = 100
bigger_test_values = (6, 18, 28, 34, 4, 26, 38, 9, 36, 39, 33, 29, 3, 16, 37, 2, 24, 22, 30, 5)
bigger_test_sizes = (21, 12, 6, 0, 34, 1, 17, 22, 4, 32, 16, 33, 8, 20, 3, 18, 39, 36, 37, 19)
num_trials = 500 # number of times that timeit will run the function

print("Recursive:", timeit.timeit('knapsack_score_recursive(capacity, bigger_test_sizes, bigger_test_values)', 
                                  number=num_trials, globals=globals()), "seconds")
print(" Memoized:", timeit.timeit('knapsack_score_memoized(capacity, bigger_test_sizes, bigger_test_values)', 
                                  number=num_trials, globals=globals()), "seconds")
print("Bottom-up:", timeit.timeit('knapsack_score_bottomup(capacity, bigger_test_sizes, bigger_test_values)', 
                                  number=num_trials, globals=globals()), "seconds")


Recursive: 12.46352070600551 seconds
 Memoized: 0.5720028169962461 seconds
Bottom-up: 0.3633419039979344 seconds


Do those timings match your expectations of how efficient these algorithms are relative to each other?  Write your thoughts in the cell below.

I predicted Bottom-up < Memoized < Recursive, runtime shortest to longest. 
It is undeniable that Recursive model would be the longest since the algorithm recalculates the same subproblems repeatedly. If the number of trials gets even larger, the time complexity will increase exponentially. 
Because the Memoized model still relies on the recursive structure, it would be less efficient thant Bottom-up model which uses less function calls and builds solutions iteratively.